### Calibration, homing and import

In [ ]:
import pandas as pd
from hardware.opentrons.http_communications import OpentronsAPI
from hardware.cameras import PendantDropCamera
from hardware.sensor.sensor_api import SensorAPI 
from hardware.opentrons.formulater import Formulater
from hardware.opentrons.configuration import Configuration
from analysis.plots import Plotter
from utils.logger import Logger
from utils.load_save_functions import load_settings
from utils.search_containers import get_well_id_concentration
from analysis.active_learning import ActiveLearner
from analysis.models import szyszkowski_model
from hardware.opentrons.droplet_manager import DropletManager
from utils.load_save_functions import save_calibration_data


opentrons_api = OpentronsAPI()
opentrons_api.initialise()
pd_cam = PendantDropCamera()
sensor_api = SensorAPI()
config = Configuration(http_api=opentrons_api)
labware = config.load_labware()
containers = config.load_containers()
pipettes = config.load_pipettes()
left_pipette = pipettes['left']
right_pipette = pipettes['right']
plotter = Plotter()
settings = load_settings()
logger = Logger(
    name="protocol",
    file_path=f'experiments/{settings["EXPERIMENT_NAME"]}/meta_data',
)

droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pd_cam,
    opentrons_api=opentrons_api,
    plotter=plotter,
    logger = logger
)

formulater = Formulater(
    left_pipette=left_pipette,
    right_pipette=right_pipette,
    containers = containers,
    logger=logger
)

2025-03-11 16:08:17,103 - INFO - Protocol already uploaded, using existing protocol.
2025-03-11 16:08:25,563 - INFO - Run created succesfully (ID: 350c9450-85ab-4155-ab28-f42d6270c432).
2025-03-11 16:08:27,198 - INFO - All custom labware definitions added.
2025-03-11 16:08:27,683 - INFO - Labware loaded successfully.
2025-03-11 16:08:31,783 - INFO - Pipettes loaded successfully.
2025-03-11 16:08:31,913 - INFO - Containers loaded successfully.
2025-03-11 16:08:52,473 - INFO - Robot homed.


In [ ]:
## Measuring script

right_pipette.pick_up_tip()
for row in ["A", "B", "C", "D", "E"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 9):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
        right_pipette.transfer(150, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6H1"], touch_tip = True, blow_out=True)   # Blank measurement
right_pipette.drop_tip()        # Takes 14.25 min for this part for 5 rows, with new touch_tip

rows = ["E", "D", "C", "B", "A"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(135-(30*i), source=containers["3A3"], destination=containers[f"6{rows[i]}1"], mix=("both", 90, 2), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

rows = ["A", "B", "C", "D", "E"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(135-(30*i), source=containers["3A2"], destination=containers[f"6{rows[i]}1"], mix=("both", 90, 2), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

rows = ["A", "B", "C", "D", "E"]
for row in range(len(rows)):
    right_pipette.pick_up_tip()
    for i in range(1, 9):
        right_pipette.transfer(150, source=containers[f"6{rows[row]}{i}"], destination=containers[f"6{rows[row]}{i+1}"], mix=("both", 50, 3), blow_out=True, touch_tip=True)
    right_pipette.drop_tip()        # approx 13 min 

for row in range(len(rows)):
    right_pipette.pick_up_tip()
    right_pipette.transfer(150, source=containers[f"6{rows[row]}8"], destination=containers["3B1"], mix=("before", 100, 3), blow_out=True)
    right_pipette.drop_tip()

2025-03-10 13:47:58,257 - INFO - Picked up tip.
2025-03-10 13:47:58,257 - INFO - Transferring 150 uL from 3A1 to well 6A1 with right pipette.
2025-03-10 13:48:08,306 - INFO - touched tip, repeated 1 times
2025-03-10 13:48:08,307 - INFO - Container: Aspirated 150 uL from this container with content MilliQ
2025-03-10 13:48:08,307 - INFO - Aspirated 150 uL from 3A1 with right pipette.
2025-03-10 13:48:13,883 - INFO - blow out done in container 6A1
2025-03-10 13:48:18,017 - INFO - touched tip, repeated 1 times
2025-03-10 13:48:18,017 - INFO - Container: Dispensed 150 uL into this container from source A1 of tube rack 15 mL (3A1) containing MilliQ
2025-03-10 13:48:18,026 - INFO - Dispensed 150 uL into well 6A1 with right pipette.
2025-03-10 13:48:18,026 - INFO - Transferring 150 uL from 3A1 to well 6A2 with right pipette.
2025-03-10 13:48:29,829 - INFO - touched tip, repeated 1 times
2025-03-10 13:48:29,829 - INFO - Container: Aspirated 150 uL from this container with content MilliQ
2025-03

In [ ]:
rows = ["A", "B", "C", "D", "E"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(135-(30*i), source=containers["3A2"], destination=containers[f"6{rows[i]}1"], mix=("both", 90, 2), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

flipped_rows = list(reversed(rows))

right_pipette.pick_up_tip()
for i in range(len(flipped_rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(135-(30*i), source=containers["3A3"], destination=containers[f"6{flipped_rows[i]}1"], mix=("both", 90, 2), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

['E', 'D', 'C', 'B', 'A']


In [3]:
right_pipette.pick_up_tip()
for row in ["A", "B"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 10):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
        right_pipette.transfer(25, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
for row in ["C", "D"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 10):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
        right_pipette.transfer(100, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
for row in ["E", "F"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 10):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
    right_pipette.transfer(150, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
for row in ["G", "H"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 10):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
        right_pipette.transfer(200, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
right_pipette.drop_tip()        # Takes 14.25 min for this part for 5 rows, with new touch_tip

2025-03-10 13:17:18,694 - INFO - Picked up tip.
2025-03-10 13:17:18,694 - INFO - Transferring 150 uL from 3A2 to well 6C1 with right pipette.
2025-03-10 13:17:28,607 - INFO - touched tip, repeated 1 times
2025-03-10 13:17:28,607 - INFO - Container: Aspirated 150 uL from this container with content blue
2025-03-10 13:17:28,607 - INFO - Aspirated 150 uL from 3A2 with right pipette.
2025-03-10 13:17:33,422 - INFO - blow out done in container 6C1
2025-03-10 13:17:37,606 - INFO - touched tip, repeated 1 times
2025-03-10 13:17:37,606 - INFO - Container: Dispensed 150 uL into this container from source A2 of tube rack 15 mL (3A2) containing blue
2025-03-10 13:17:37,606 - INFO - Dispensed 150 uL into well 6C1 with right pipette.
2025-03-10 13:17:50,750 - INFO - Right pipette dropped tip into trash.


In [2]:
stock_c12e4 = containers["3A3"]

stock_ctab = containers["3A2"]
well = containers["6B2"]

# before mixes 4x total, 2x before asp, 2x before disp
# both mixes 8x total 2x before asp, 
#   2x after asp, move, 2x before disp, 2x after disp
# after mixes 4x total, 2x after asp, 2x after disp


right_pipette.pick_up_tip()
# right_pipette.transfer(150, source=containers["3A1"], destination=containers["6A3"], blow_out=True) # mixes 4x total, 
# right_pipette.transfer(150, source=containers["3A1"], destination=containers["6A2"])   
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6A2"], mix=("before", 40, 2))  # mixes 4x total
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6A2"], mix=("both", 40, 2))
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6A2"], mix=("after", 40, 2))

2025-03-05 15:02:52,389 - INFO - Picked up tip.
2025-03-05 15:02:52,389 - INFO - Transferring 150 uL from 3A1 to well 6A2 with right pipette.
2025-03-05 15:02:56,960 - INFO - done with mixing in 3A1 with order before, with volume 40, repeated 2 times
2025-03-05 15:02:58,960 - INFO - Container: Aspirated 150 uL from this container with content water
2025-03-05 15:02:58,960 - INFO - Aspirated 150 uL from 3A1 with right pipette.
2025-03-05 15:03:03,183 - INFO - Container: Dispensed 150 uL into this container from source A1 of tube rack 15 mL (3A1) containing water
2025-03-05 15:03:03,185 - INFO - Dispensed 150 uL into well 6A2 with right pipette.
2025-03-05 15:03:03,186 - INFO - Transferring 150 uL from 3A1 to well 6A2 with right pipette.
2025-03-05 15:03:08,660 - INFO - done with mixing in 3A1 with order both, with volume 40, repeated 2 times
2025-03-05 15:03:10,670 - INFO - Container: Aspirated 150 uL from this container with content water
2025-03-05 15:03:10,671 - INFO - Aspirated 150 

In [ ]:
well = containers["6A2"]
well.concentration = 4
if well.concentration not in ["pure", "empty"]:
    print("reached")

reached


### Reset index (Clean pipettes return works)

In [3]:
right_pipette.well_index = 0
if right_pipette.has_tip == True:
    right_pipette.drop_tip()	


2025-03-11 16:55:01,803 - INFO - Right pipette dropped tip into trash.


In [ ]:
#offset = dict(x=-0.6, y=1.1, z=0.6)
#left_pipette.move_to_tip(well="H12", offset=offset)

In [ ]:
if not right_pipette.has_tip:
	right_pipette.pick_up_tip()
right_pipette.drop_tip(return_tip=True)

2025-02-24 11:39:41,769 - INFO - right pipette (p1000_single_gen2) returned tip to well B1 on tips P1000, 1.


In [2]:
### Test script for flow rate
rows = ["A", "B", "C", "D", "E"]
right_pipette.pick_up_tip()
for row in range(len(rows)):     # Loop to fill with 150 ul of water in all used wells
    for i in range(2, 9):               # By default blowout occurs before touch tip
        right_pipette.transfer(150, source=containers["3A1"], destination=containers[f"6{rows[row]}{i}"], touch_tip = True, blow_out=True, flow_rate=(100-(20*row)))
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6H1"], touch_tip = True, blow_out=True, flow_rate=(100-(20*row)))   # Blank measurement
# right_pipette.drop_tip()        # Takes 14.25 min for this part for 5 rows, with new touch_tip

# right_pipette.pick_up_tip()
for row in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6{rows[row]}1"], mix=("both", 90, 2), blow_out=True, touch_tip=True, flow_rate=(100-(20*row)))
    right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6{rows[row]}2"], mix=("both", 90, 2), blow_out=True, touch_tip=True,flow_rate=(100-(20*row)))
# right_pipette.drop_tip()

# right_pipette.pick_up_tip()
for row in range(len(rows)):
    for i in range(2, 9):
        right_pipette.transfer(150, source=containers[f"6{rows[row]}{i}"], destination=containers[f"6{rows[row]}{i+1}"], mix=("both", 50, 3), blow_out=True, touch_tip=True, flow_rate=(100-(20*row)))
# right_pipette.drop_tip()        # approx 13 min 

# right_pipette.pick_up_tip()
for row in range(len(rows)):
    right_pipette.transfer(150, source=containers[f"6{rows[row]}8"], destination=containers[f"6{rows[row]}9"], mix=("before", 100, 3), blow_out=True, flow_rate=(100-(20*row)))
right_pipette.drop_tip()

2025-03-11 16:09:16,323 - INFO - Picked up tip.
2025-03-11 16:09:16,323 - INFO - Transferring 150 uL from 3A1 to well 6A2 with right pipette.


100


2025-03-11 16:09:25,713 - INFO - touched tip, repeated 1 times
2025-03-11 16:09:25,723 - INFO - Container: Aspirated 150 uL from this container with content MilliQ
2025-03-11 16:09:25,723 - INFO - Aspirated 150 uL from 3A1 with right pipette.


100


2025-03-11 16:09:31,777 - INFO - blow out done in container 6A2
2025-03-11 16:09:36,126 - INFO - touched tip, repeated 1 times
2025-03-11 16:09:36,126 - INFO - Container: Dispensed 150 uL into this container from source A1 of tube rack 15 mL (3A1) containing MilliQ
2025-03-11 16:09:36,126 - INFO - Dispensed 150 uL into well 6A2 with right pipette.
2025-03-11 16:09:36,141 - INFO - Transferring 150 uL from 3A1 to well 6A3 with right pipette.


...

2025-03-11 16:54:33,046 - INFO - Container: Aspirated 150 uL from this container with content nan
2025-03-11 16:54:33,046 - INFO - Aspirated 150 uL from 6B8 with right pipette.


80
80


2025-03-11 16:54:37,078 - INFO - blow out done in container 6B9
2025-03-11 16:54:37,093 - INFO - Container: Dispensed 150 uL into this container from source B8 of plate 1 (6B8) containing nan
2025-03-11 16:54:37,094 - INFO - Dispensed 150 uL into well 6B9 with right pipette.
2025-03-11 16:54:37,097 - INFO - Transferring 150 uL from 6C8 to well 6C9 with right pipette.


40


KeyboardInterrupt: 

### TEST SCRIPT FOR pipette tips reuse

In [2]:
right_pipette.pick_up_tip()
for row in ["A", "B"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 9):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
        right_pipette.transfer(125, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6H1"], touch_tip = True, blow_out=True)   # Blank measurement

right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6A1"], blow_out=True, mix=("after", 100, 2), touch_tip=True)
right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6B1"], blow_out=True, mix=("after", 100, 2), touch_tip=True)
right_pipette.drop_tip()

for i in range(1, 9):
    right_pipette.pick_up_tip()
    right_pipette.transfer(150, source=containers[f"6A{i}"], destination=containers[f"6A{i+1}"], mix=("before", 100, 3), touch_tip=True, blow_out=True)
    right_pipette.drop_tip()

right_pipette.pick_up_tip()
for i in range(1, 9):
    right_pipette.transfer(150, source=containers[f"6B{i}"], destination=containers[f"6B{i+1}"], mix=("before", 100, 3), touch_tip=True, blow_out=True)
right_pipette.drop_tip()

2025-03-10 12:53:09,309 - INFO - Picked up tip.
2025-03-10 12:53:09,309 - INFO - Transferring 125 uL from 3A1 to well 6A1 with right pipette.
2025-03-10 12:53:18,458 - INFO - touched tip, repeated 1 times
2025-03-10 12:53:18,467 - INFO - Container: Aspirated 125 uL from this container with content MilliQ
2025-03-10 12:53:18,467 - INFO - Aspirated 125 uL from 3A1 with right pipette.
2025-03-10 12:53:23,007 - INFO - blow out done in container 6A1
2025-03-10 12:53:26,987 - INFO - touched tip, repeated 1 times
2025-03-10 12:53:26,988 - INFO - Container: Dispensed 125 uL into this container from source A1 of tube rack 15 mL (3A1) containing MilliQ
2025-03-10 12:53:26,988 - INFO - Dispensed 125 uL into well 6A1 with right pipette.
2025-03-10 12:53:26,988 - INFO - Transferring 125 uL from 3A1 to well 6A2 with right pipette.
2025-03-10 12:53:38,099 - INFO - touched tip, repeated 1 times
2025-03-10 12:53:38,100 - INFO - Container: Aspirated 125 uL from this container with content MilliQ
2025-03

### Fully Functional Vertical Dilution (From a single high concentration down) Much faster


In [ ]:
right_pipette.pick_up_tip()
for row in ["A", "B", "C", "D", "E", "F"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 9):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
        right_pipette.transfer(125, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6H1"], touch_tip = True, blow_out=True)   # Blank measurement
right_pipette.drop_tip()        # Takes 9.5 min for this part for 7 rows

rows = ["A", "B", "C", "D", "E", "F"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(150-(25*i), source=containers["3A2"], destination=containers[f"6{rows[i]}1"], blow_out=True, mix=("after", 40, 2), touch_tip=True)
right_pipette.drop_tip()    # takes 2.25 min

rows = ["F", "E", "D", "C", "B", "A"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(150-(25*i), source=containers["3A3"], destination=containers[f"6{rows[i]}1"], blow_out=True, mix=("after", 40, 2), touch_tip=True)
right_pipette.drop_tip()

rows = ["A", "B", "C", "D", "E", "F"]
for row in range(len(rows)):
    right_pipette.pick_up_tip()
    for i in range(1, 9):
        right_pipette.transfer(150, source=containers[f"6{rows[row]}{i}"], destination=containers[f"6{rows[row]}{i+1}"], mix=("after", 50, 3), touch_tip=True, blow_out=True)
    right_pipette.drop_tip()        # approx 13 min 
    #TODO write trash script, so all amounts are equal?


2025-02-26 10:39:24,469 - INFO - right pipette (p1000_single_gen2) picked up tip from well A1 on tips P1000, 1.
2025-02-26 10:39:24,469 - INFO - Transferring 125 uL from water (well A1 on tube rack 15 mL) to well A1 on plate 1 with right pipette (p1000_single_gen2)
2025-02-26 10:39:24,484 - WARNING - right pipette (p1000_single_gen2) is not clean! Aspirating anyway...
2025-02-26 10:39:31,284 - INFO - touched tip, repeated 1 times
2025-02-26 10:39:31,284 - INFO - Container: Aspirated 125 uL from this container with content water
2025-02-26 10:39:31,284 - INFO - Aspirated 125 uL from water (well A1 on tube rack 15 mL) with right pipette (p1000_single_gen2)
2025-02-26 10:39:35,884 - INFO - blow out done in container 6A1
2025-02-26 10:39:37,864 - INFO - touched tip, repeated 1 times
2025-02-26 10:39:37,864 - ERROR - Overflowing of container!
2025-02-26 10:39:37,864 - INFO - Dispensed 125 uL into well 6A1 with right pipette (p1000_single_gen2)
2025-02-26 10:39:37,864 - INFO - Transferring 1

### Dilution code according to READ_ME

### Horizontal Dilution (Sideways dilution using rows of diluted pure surfactant)

In [ ]:
# A1+B1=C1, A2+B2=C2, A3+B3=C3 Dilution
right_pipette.pick_up_tip()
for row in ["A", "B", "C", "D", "E", "F", "G"]:    # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 9):                             # By default blowout occurs before touch tip
        right_pipette.transfer(150, source=containers["3A1"], destination=containers[f"6{row}{col}"], touch_tip = True, blow_out=True)
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6H1"], touch_tip = True, blow_out=True)   # Blank measurement
right_pipette.drop_tip()                  # Not really necessary, but useful to avoid tip contamination

#  Initial plate stock of Phenol Red and Methylene Blue
right_pipette.pick_up_tip()
right_pipette.transfer(150, source=containers["3A3"], destination=containers[f"6G1"], touch_tip=True, blow_out=True)    # MBlue should be in A3
right_pipette.drop_tip()

right_pipette.pick_up_tip()
right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6A1"], touch_tip=True, blow_out=True)    # PRed should be in A2
right_pipette.drop_tip() 

# Initial dilution of Phenol Red and Methylene Blue
for row in ["A","G"]: 
    right_pipette.pick_up_tip()
    for i in range(1, 9):
        right_pipette.transfer(150, source=containers[f"6{row}{i}"], destination=containers[f"6{row}{i+1}"], mix=("after", 40, 2), touch_tip=True, blow_out=True)
    right_pipette.drop_tip()                  # ONLY POSSIBLE BECAUSE WE USE PHENOL RED AGAIN, CHANGE OTHERWISE TODO

# Seeding all mixed wells with Phenol Red
rows = ["A", "B", "C", "D", "E", "F", "G"]
for col in range(1, 9):
    right_pipette.pick_up_tip()
    for i in range(len(rows) - 2):  # Loop through rows, but stop before the last one
        right_pipette.transfer(50-(10*i), source=containers[f"6A{col}"], destination=containers[f"6{rows[i + 1]}{col}"], blow_out=True, mix=("after", 40, 2), touch_tip=True)
    right_pipette.drop_tip()

#  Initial dilution of Methylene Blue
rows = ["H", "G", "F", "E", "D", "C", "B"]      # Doesnt deposit in row B
for col in range(1, 9):
    right_pipette.pick_up_tip()
    for i in range(len(rows) - 2):  # Loop through rows, but stop before the last one
        right_pipette.transfer(50-(10*i), source=containers[f"6G{col}"], destination=containers[f"6{rows[i + 1]}{col}"], blow_out=True, mix=("after", 40, 2), touch_tip=True)
    right_pipette.drop_tip()

2025-02-13 15:29:32,799 - INFO - right pipette (p1000_single_gen2) picked up tip from well A1 on tips P1000, 1.
2025-02-13 15:29:58,179 - INFO - Robot homed.
2025-02-13 15:29:58,180 - INFO - Transferring 150 uL from water (well A1 on tube rack 15 mL) to well C2 on plate 1 with right pipette (p1000_single_gen2)
2025-02-13 15:30:04,726 - INFO - touched tip, repeated 1 times
2025-02-13 15:30:04,727 - INFO - Container: Aspirated 150 uL from this container with content water
2025-02-13 15:30:04,729 - INFO - Aspirated 150 uL from water (well A1 on tube rack 15 mL) with right pipette (p1000_single_gen2)
2025-02-13 15:30:09,538 - INFO - blow out done in container 6C2
2025-02-13 15:30:11,529 - INFO - touched tip, repeated 1 times
2025-02-13 15:30:11,530 - INFO - Container: Dispensed 150 uL into this container from source A1 of tube rack 15 mL (3A1) containing water
2025-02-13 15:30:11,533 - INFO - Dispensed 150 uL into well 6C2 with right pipette (p1000_single_gen2)
2025-02-13 15:30:11,534 - IN

In [ ]:
# # Seeding all mixed wells with Phenol Red
# rows = ["A", "B", "C", "D", "E", "F", "G"]
# for i in range(len(rows) - 2):  # Loop through rows, but stop before the last one
#     print(rows[i+1])

#  Initial dilution of Methylene Blue
rows = ["G", "F", "E", "D", "C", "B",  "A"]      # Doesnt deposit in row B
for i, row in enumerate(rows):
    print(row)

G
F
E
D
C
B
A


### TAKE OFF CAPS BEFORE RUNNING ABOVE

In [ ]:
right_pipette.pick_up_tip()
for i in range(3, 0, -1):
    right_pipette.transfer(50, source=containers[f"6B{i}"], destination=containers[f"6C{i+1}"],  touch_tip = True)
right_pipette.drop_tip()

right_pipette.pick_up_tip()
for i in range(3, 0, -1):
    right_pipette.transfer(50, source=containers[f"6A{i}"], destination=containers[f"6C{i+1}"],  touch_tip = True)

for i in range (1, 4):
    containers[f"6"] = f"6C{i}"
right_pipette.drop_tip()

In [ ]:
right_pipette.pick_up_tip()
for row in ["A", "B", "C"]:  # Loop to fill with 100 ul of water in all used wells
    for col in range(1, 4):
        well = f"6{row}{col}"
        right_pipette.transfer(100, source=containers["3A1"], destination=containers[well])
right_pipette.drop_tip() 

In [ ]:
# right_pipette.pick_up_tip()
# for row in ["A", "B", "C"]:  # Loop to fill with 100 ul of water in all used wells
    # for col in range(1, 4):
        # well = f"6{row}{col}"
        # right_pipette.transfer(100, source=containers["3A3"], destination=containers[well])
# right_pipette.drop_tip()


right_pipette.pick_up_tip()
for i in range(1, 4):
    right_pipette.transfer(100, source=containers[f"6A{i}"], destination=containers[f"6A{i+1}"])
right_pipette.drop_tip()

right_pipette.pick_up_tip()
for i in range(1, 4):
    right_pipette.transfer(100, source=containers[f"6B{i}"], destination=containers[f"6B{i+1}"])
right_pipette.drop_tip()

right_pipette.pick_up_tip()
for i in range(1, 4):
    right_pipette.transfer(100, source=containers[f"6C{i}"], destination=containers[f"6C{i+1}"])
right_pipette.drop_tip()

In [6]:
right_pipette.pick_up_tip()
for row in ["A", "B", "C", "D"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(2, 6):               # By default blowout occurs before touch tip
        well = f"6{row}{col}"
        right_pipette.transfer(150, source=containers["3A1"], destination=containers[well], touch_tip = True, blow_out=True)
right_pipette.transfer(150, source=containers["3A1"], destination=containers["6H1"], touch_tip = True, blow_out=True)   # Blank measurement
right_pipette.drop_tip()        # Takes 14.25 min for this part for 5 rows, with new touch_tip


right_pipette.pick_up_tip()
right_pipette.transfer(150, source=containers["3A3"], destination=containers[f"6A1"], mix=("both", 40, 2), blow_out=True, touch_tip=True)
right_pipette.transfer(150, source=containers["3A3"], destination=containers[f"6A2"], mix=("both", 40, 2), blow_out=True, touch_tip=True)
right_pipette.transfer(150, source=containers["3A3"], destination=containers[f"6C1"], mix=("both", 80, 3), blow_out=True, touch_tip=True)
right_pipette.transfer(150, source=containers["3A3"], destination=containers[f"6C2"], mix=("both", 80, 3), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

right_pipette.pick_up_tip()
right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6B1"], mix=("both", 40, 2), blow_out=True, touch_tip=True)
right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6B2"], mix=("both", 40, 2), blow_out=True, touch_tip=True)
right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6D1"], mix=("both", 80, 3), blow_out=True, touch_tip=True)
right_pipette.transfer(150, source=containers["3A2"], destination=containers[f"6D2"], mix=("both", 80, 3), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

rows = ["A", "B"]
for row in range(len(rows)):
    right_pipette.pick_up_tip()
    for i in range(2, 6):
        right_pipette.transfer(150, source=containers[f"6{rows[row]}{i}"], destination=containers[f"6{rows[row]}{i+1}"], mix=("both", 50, 3), blow_out=True, touch_tip=True)
    right_pipette.drop_tip()        # approx 13 min 

rows = ["C", "D"]
for row in range(len(rows)):
    right_pipette.pick_up_tip()
    for i in range(2, 6):
        right_pipette.transfer(150, source=containers[f"6{rows[row]}{i}"], destination=containers[f"6{rows[row]}{i+1}"], mix=("both", 100, 3), blow_out=True, touch_tip=True)
    right_pipette.drop_tip()   

# for row in range(len(rows)):
#     right_pipette.pick_up_tip()
#     right_pipette.transfer(150, source=containers[f"6{rows[row]}8"], destination=containers["3B1"], mix=("before", 50, 3), blow_out=True)
#     right_pipette.drop_tip()

2025-03-06 14:48:21,816 - INFO - Picked up tip.
2025-03-06 14:48:21,816 - INFO - Transferring 150 uL from 3A1 to well 6A2 with right pipette.
2025-03-06 14:48:31,459 - INFO - touched tip, repeated 1 times
2025-03-06 14:48:31,459 - INFO - Container: Aspirated 150 uL from this container with content water
2025-03-06 14:48:31,459 - INFO - Aspirated 150 uL from 3A1 with right pipette.
2025-03-06 14:48:36,467 - INFO - blow out done in container 6A2
2025-03-06 14:48:40,682 - INFO - touched tip, repeated 1 times
2025-03-06 14:48:40,683 - INFO - Container: Dispensed 150 uL into this container from source A1 of tube rack 15 mL (3A1) containing water
2025-03-06 14:48:40,683 - INFO - Dispensed 150 uL into well 6A2 with right pipette.
2025-03-06 14:48:40,683 - INFO - Transferring 150 uL from 3A1 to well 6A3 with right pipette.
2025-03-06 14:48:52,116 - INFO - touched tip, repeated 1 times
2025-03-06 14:48:52,117 - INFO - Container: Aspirated 150 uL from this container with content water
2025-03-06